In [ ]:
# !pip install -q transformers
! pip install -U accelerate
! pip install -U transformers 

In [ ]:
!pip install -q datasets jiwer
!pip install gdown

In [ ]:
# !gdown 1lR0b1QBIsXk9JqL9__HgJQi0PdWeNxKi
!gdown 1b4fCTrnfKnR0GHm1XCve9nhy7JyrqahR

In [ ]:
# !unzip -q /kaggle/working/training_data.zip


In [ ]:
import pandas as pd

df = pd.read_fwf('/kaggle/working/train_gt.txt', header=None)
# df.rename(columns={0: "file_name", 1: "text"}, inplace=True)

# df['file_name'] = df['file_name'].apply(lambda x: x + 'g' if x.endswith('jp') else x)
for index, row in df.iterrows():
  input = row
  input = input[0].split()
  df.at[index, 0] = input[0]  # Assign the first part to the first column
  df.at[index, 1] = ' '.join(input[1:])  # Assign the rest to the second column
# print(df['file_name'][26909])
df.rename(columns={0: "file_name", 1: "text"}, inplace=True)

df.head()

In [ ]:
# prompt: i want to see a few example image in /content/new_train print it to see about 2 image

import matplotlib.pyplot as plt
import os

# Get the path to the new_train directory
new_train_path = "/kaggle/working/new_train"

# Get a list of all files in the new_train directory
files = os.listdir(new_train_path)

# Select two random files from the list

# Load and display the images
i = 0
for file in files:
    print(file)
    image_path = os.path.join(new_train_path, file)
    image = plt.imread(image_path)
    plt.imshow(image)
    plt.show()
    i += 1
    if i == 2:
      break


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
eval_df, test_df = train_test_split(test_df, test_size=0.5)
train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class VOCRDataset(Dataset):
    def __init__(self, root_dir, df, processor, transform, tokenizer, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.transform = transform
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        # labels = self.processor.tokenizer(text,
        #                                   padding="max_length",
        #                                   max_length=self.max_target_length).input_ids
        labels = self.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        # labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
from transformers import AutoImageProcessor, ViTModel
from torchvision import transforms
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

train_dataset = VOCRDataset(root_dir='/kaggle/working/new_train/',
#                            df=train_df,                                                       
                            df=test_df,
                           processor=processor,
                            transform = transform,
                            tokenizer=tokenizer)
eval_dataset = VOCRDataset(root_dir='/kaggle/working/new_train/',
                           df=eval_df,
                           processor=processor,
                            transform = transform,
                           tokenizer=tokenizer)
test_dataset = VOCRDataset(root_dir='/kaggle/working/new_train/',
                           df=test_df,
                           processor=processor,
                            transform = transform,
                           tokenizer=tokenizer)



In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))
print("Number of test examples:", len(test_dataset))

In [ ]:
print(type(train_dataset))
print(train_dataset)
encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

In [ ]:
image = Image.open(train_dataset.root_dir + train_df['file_name'][0]).convert("RGB")
image

In [ ]:
labels = encoding['labels']
labels[labels == -100] = tokenizer.pad_token_id
label_str = tokenizer.decode(labels, skip_special_tokens=True)
print(label_str)

In [ ]:

from transformers import VisionEncoderDecoderModel


encoder_checkpoint = "google/vit-base-patch16-224-in21k"
decoder_checkpoint = "vinai/phobert-base"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_pretrained_model_name_or_path=encoder_checkpoint,
    decoder_pretrained_model_name_or_path=decoder_checkpoint,
)

# ensure that randomly initialized cross-attention layers are added
assert model.config.decoder.is_decoder is True
assert model.config.decoder.add_cross_attention is True

# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
# model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("google/vit-base-patch16-224-in21k", "aubmindlab/bert-base-arabertv2")


In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    output_dir="./",
    logging_steps=2,
    save_steps=5150,
    eval_steps=5150,
    report_to="none",
)

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
import wandb
wandb.init(mode="disabled")


# os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
from transformers import default_data_collator
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()